In [18]:
import pandas as pd
import numpy as np
from collections import Counter
from vincenty import vincenty

In [19]:
colnames = ['lat','long', 'null', 'alt' ,'DateTime','Date','Time']
test = pd.read_csv('/Users/williamyee/Documents/Data Bootcamp/unsupervised-learning-metis/Geolife Trajectories 1.3/Data/013/Trajectory/20080929001436.plt', skiprows=6, names = colnames)

In [20]:
test.head()

,lat,long,null,alt,DateTime,Date,Time
0,39.647024,118.161976,0,492,39720.010139,2008-09-29,00:14:36
1,39.647015,118.161972,0,492,39720.010174,2008-09-29,00:14:39
2,39.647052,118.161963,0,492,39720.010208,2008-09-29,00:14:42
3,39.647115,118.161961,0,492,39720.010243,2008-09-29,00:14:45
4,39.647114,118.161939,0,492,39720.010278,2008-09-29,00:14:48


In [21]:

# add some initial shifts
test['lat_shift'] = test.lat.shift(-1)
test['long_shift'] = test.long.shift(-1)
test['time_shift'] = test.DateTime.shift(-1)

def speed(x):
    try:
        s = vincenty((x[-3],x[-2]),(x[0],x[1]), miles = True) / ((x[-1]-x[4]) * 24)
    except:
        s= np.nan
    return s
test['speed_mph'] = test.apply(speed,axis =1)
test['speed_shift'] = test.speed_mph.shift(-1)

def accel(x):
    try:
        a = (x[-1] - x[-2]) / ((x[9] - x[4]) *24)
    except:
        a = np.nan
    return a
test['acceleration'] = test.apply(accel, axis =1)
test['acceleration_shift'] = test.acceleration.shift(-1)

def jerk(x):
    try:
        j = (x[-1] - x[-2]) / ((x[9] - x[4]) *24)
    except:
        j = np.nan
    return a
test['jerk'] = test.apply(accel, axis =1)
test['jerk_shift'] = test.jerk.shift(-1)

def y(x):
    try:
        yy = np.sin((x[8] - x[0]) * np.pi/180) *np.cos( x[7]* np.pi/180)
    except:
        yy= np.nan
    return yy
test['y'] = test.apply(y, axis =1)

def x(x):
    try:
        xx = np.cos(x[0] * np.pi/180) *np.sin(x[7]* np.pi/180) - np.sin(x[0]* np.pi/180) * np.cos(x[7]* np.pi/180)*np.cos((x[8]-x[1])* np.pi/180)
    except:
        xx = np.nan
    return xx
test['x'] = test.apply(x,axis =1)

def bearing(x):
    try:
        b = np.arctan2(x[-2],x[-1])*180/np.pi
    except:
        b = np.nan
    return b
test['bearing'] = test.apply(bearing,axis=1)

test['brearing_shift'] = test.bearing.shift(-1)

def bearing_rate(x):
    try:
        br = abs(x[-1]-x[-2])
    except:
        br = np.nan
    return br
test['bearing_rate'] = test.apply(bearing_rate,axis=1)

In [34]:
test.head()

,lat,long,null,alt,DateTime,Date,Time,lat_shift,long_shift,time_shift,...,speed_shift,acceleration,acceleration_shift,jerk,jerk_shift,y,x,bearing,brearing_shift,bearing_rate
0,39.647024,118.161976,0,492,39720.010139,2008-09-29,00:14:36,39.647015,118.161972,39720.010174,...,3.116401,2793.602981,2521.426247,-3.266123e+05,-8.508630e+06,0.754572,-1.570796e-07,90.000012,89.999951,0.000061
1,39.647015,118.161972,0,492,39720.010174,2008-09-29,00:14:39,39.647052,118.161963,39720.010208,...,5.217589,2521.426247,-4569.096021,-8.508630e+06,2.203367e+07,0.754572,6.457718e-07,89.999951,89.999917,0.000034
2,39.647052,118.161963,0,492,39720.010208,2008-09-29,00:14:42,39.647115,118.161961,39720.010243,...,1.410001,-4569.096021,13792.336673,2.203367e+07,-3.237587e+07,0.754571,1.099557e-06,89.999917,90.000001,0.000085
3,39.647115,118.161961,0,492,39720.010243,2008-09-29,00:14:45,39.647114,118.161939,39720.010278,...,12.903608,13792.336673,-13187.535942,-3.237587e+07,3.221171e+07,0.754571,-1.745326e-08,90.000001,90.000200,0.000199
4,39.647114,118.161939,0,492,39720.010278,2008-09-29,00:14:48,39.646963,118.161889,39720.010313,...,1.914001,-13187.535942,13655.539850,3.221171e+07,-3.158619e+07,0.754572,-2.635447e-06,90.000200,89.999973,0.000227


In [35]:
test.tail()

,lat,long,null,alt,DateTime,Date,Time,lat_shift,long_shift,time_shift,...,speed_shift,acceleration,acceleration_shift,jerk,jerk_shift,y,x,bearing,brearing_shift,bearing_rate
1264,39.643961,118.167074,0,115,39720.120313,2008-09-29,02:53:15,39.644008,118.167071,39720.120347,...,2.931602,-1156.320661,-476.650335,8.156049e+05,-1.964710e+06,0.754627,8.203047e-07,89.999938,89.999954,0.000016
1265,39.644008,118.167071,0,115,39720.120347,2008-09-29,02:53:18,39.644043,118.167078,39720.120382,...,2.534394,-476.650335,-2113.907432,-1.964710e+06,NaN,0.754626,6.108652e-07,89.999954,89.999963,0.000009
1266,39.644043,118.167078,0,115,39720.120382,2008-09-29,02:53:21,39.644071,118.167094,39720.120417,...,0.772800,-2113.907432,NaN,NaN,NaN,0.754626,4.886922e-07,89.999963,89.999999,0.000036
1267,39.644071,118.167094,0,115,39720.120417,2008-09-29,02:53:24,39.644072,118.167106,39720.120451,...,NaN,NaN,NaN,NaN,NaN,0.754626,1.745330e-08,89.999999,NaN,NaN
1268,39.644072,118.167106,0,115,39720.120451,2008-09-29,02:53:27,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
test.describe()

,lat,long,null,alt,DateTime,lat_shift,long_shift,time_shift,speed_mph,speed_shift,acceleration,acceleration_shift,jerk,jerk_shift,y,x,bearing,brearing_shift,bearing_rate
count,1269.000000,1269.000000,1269.0,1269.000000,1269.000000,1268.000000,1268.000000,1268.000000,1268.000000,1267.000000,1267.000000,1266.000000,1.266000e+03,1.265000e+03,1268.000000,1.268000e+03,1268.000000,1267.000000,1267.000000
mean,39.648052,118.163840,0.0,181.128448,39720.060357,39.648053,118.163842,39720.060397,2.461536,2.462857,112.798747,110.681208,-1.726836e+05,-1.725619e+05,0.754563,-4.063243e-08,90.000003,90.000003,0.000042
std,0.001444,0.002056,0.0,104.006188,0.040814,0.001444,0.002056,0.040806,4.287499,4.288934,5689.159668,5690.908385,1.129616e+07,1.130063e+07,0.000024,2.790735e-06,0.000212,0.000212,0.000296
min,39.643777,118.160667,0.0,-118.000000,39720.010139,39.643777,118.160667,39720.010174,0.000000,0.000000,-74810.534305,-74810.534305,-1.985705e+08,-1.985705e+08,0.754522,-4.176572e-05,89.993539,89.993539,0.000000
25%,39.647379,118.162108,0.0,116.000000,39720.023299,39.647383,118.162110,39720.023325,1.024800,1.025399,-856.082461,-857.162666,-1.815688e+06,-1.817842e+06,0.754546,-2.967060e-07,89.999985,89.999985,0.000005
50%,39.648362,118.163518,0.0,147.000000,39720.039688,39.648362,118.163520,39720.039705,1.923598,1.923601,-37.440106,-38.160076,4.794151e+04,4.925144e+04,0.754558,-3.490658e-08,90.000003,90.000003,0.000011
75%,39.649118,118.165009,0.0,226.000000,39720.104132,39.649123,118.165010,39720.104141,2.935800,2.936397,1053.361014,1050.120880,2.095636e+06,2.096068e+06,0.754574,1.919862e-07,90.000023,90.000023,0.000025
max,39.651702,118.168604,0.0,492.000000,39720.120451,39.651702,118.168604,39720.120451,99.419925,99.419925,106172.605848,106172.605848,1.422704e+08,1.422704e+08,0.754630,8.508490e-05,90.003171,90.003171,0.006636
